# Contextualized Prompts 
This tutorial guides you through creating an LLM generated restaurant recommendation function.
This is an example of how Tecton managed and contextualized prompts enable personalization.

It uses Tecton's real-time enriched prompts to provide current context to the LLM in order to improve the quality of its response. 
This tutorial demonstrates both LangChain and LlamaIndex integration with Tecton prompts.

## Install Packages

In [45]:
!pip install 'tecton[rift]==0.10.0b32' gcsfs s3fs --force-reinstall
!pip install "$HOME/Downloads/tecton_utils-0.0.4-py3-none-any.whl" --force-reinstall

!pip install openai
!pip install langchain-openai
!pip install langchain
!pip install langchain_community
!pip install langchain_core

  Using cached tecton-0.10.0b32-py3-none-any.whl.metadata (6.8 kB)
  Using cached gcsfs-2024.6.1-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached s3fs-2024.6.1-py3-none-any.whl.metadata (1.6 kB)
  Using cached attrs-24.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached numpy-1.26.4-cp39-cp39-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached pathspec-0.12.1-py3-none-any.whl.metadata (21 kB)
  Using cached pendulum-2.1.2-cp39-cp39-macosx_14_0_arm64.whl
  Using cached protobuf-5.27.3-cp38-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached pytimeparse-1.1.8-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached pandas-2.2.2-cp39-cp39-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached texttable-1.7.0-py2.py3-none-any.whl.metadata (9.8 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.

# Tecton Prompt

In the following cell you'll create a Tecton Agent with a system prompt that provides instructions to the LLM. The instructions are parameterized with a specific user's data. 

The agent creation function takes a Tecton feature view as input which is used at run-time to acquire the latest values of the parameters for the user.

In [39]:
from tecton_utils.ai import AgentClient, AgentService, prompt
from tecton_utils.core_utils import make_request_source



def restaurant_recommender_agent( user_info):    

    location_request = make_request_source(location = str)
     
    @prompt(sources=[user_info])
    def sys_prompt(location: str, user_info):
        name = user_info["name"]
        food_preference = user_info["food_preference"]
        return f"""
        You are a consierge service that recommends restaurants.
        You are serving {name}. Address them by name. 
        Respond to the user query about dining. 
        If the user asks for a restaurant recommendation respond with a specific restaurant that you know and suggested menu items. 
        Suggest restaurants that are in {location}. 
        If the user does not provide a cuisine or food preference, choose a {food_preference} restaurant.
        """
    @prompt(sources=[ location_request, user_info])
    def sys_prompt_request(location_request, user_info ):
        name = user_info["name"]
        food_preference = user_info["food_preference"]
        location = location_request["location"]
        return f"""
        You are a consierge service that recommends restaurants.
        You are serving {name}. Address them by name. 
        Respond to the user query about dining. 
        If the user asks for a restaurant recommendation respond with a specific restaurant that you know and suggested menu items. 
        Suggest restaurants that are in {location}. 
        If the user does not provide a cuisine or food preference, choose a {food_preference} restaurant.
        """
        
    return AgentService(
        name="restaurant_recommender",
        prompts=[sys_prompt, sys_prompt_request],
        tools=[user_info],
    )

The example above uses a single feature view as input. Tecton Agents can make use of any number of feature views deployed on the Tecton platform to provide up to date context from any features deployed on the platform. 

Notice that the `sys_prompt` function additionally takes the `location` parameter in the prompt. This instructs Tecton to acquire the location information at request time. Location is a good example of a real-time input given that it would presumably come from a device's GPS function. A combination of existing feature pipelines and real-time parameters can be used for any prompt.

## Sample Data

In order to keep this notebook self-contained, you will create a mock feature view with some hard-coded data.
In a real application, you would use Feature Views that continuously update feature values and therefore provide up-to-date context to the LLM application.

In [40]:
import pandas as pd
from tecton import RequestSource
from tecton.types import Field, String


from tecton_utils.testing import mock_batch_feature_view


mock_data = pd.DataFrame(
        [
            {
                "user_id": "user1",
                "name": "Jim",
                "age": 30,
                "food_preference": "American",
            },
            {
                "user_id": "user2",
                "name": "John",
                "age": 40,
                "food_preference": "Italian",
            },
            {
                "user_id": "user3",
                "name": "Jane",
                "age": 50,
                "food_preference": "Chinese",
            },
        ]
    )

user_preference_fv = mock_batch_feature_view(
        "user_info", mock_data, entity_keys=["user_id"], description="User's profile with name, age and food preference."
    )

The feature view identifies the key `user_id` that is needed to access a user's data, this attribute must be provided when using the feature view in a prompt. 

In the following cell, you will test the prompt through an AgentClient's invoke_prompt method using a `user_id` and a `location` value. The `user_id` is used to retrieve a specific user's values. The location parameter is a request time parameter so you'll need to provide that value too.

In [41]:
# create the Tecton Agent
recommender_agent = restaurant_recommender_agent(user_preference_fv )

# create a client to invoke with the agent
client = AgentClient.from_local( recommender_agent )

#test the agent using "sys_prompt" prompt
print(client.invoke_prompt("sys_prompt_request", user_id="user3", location="Chicago"))



        You are a consierge service that recommends restaurants.
        You are serving Jane. Address them by name. 
        Respond to the user query about dining. 
        If the user asks for a restaurant recommendation respond with a specific restaurant that you know and suggested menu items. 
        Suggest restaurants that are in Chicago. 
        If the user does not provide a cuisine or food preference, choose a Chinese restaurant.
        


## Incorporate Contextualized Prompt into a LangChain agent

The Tecton AgentClient can be used to create a LangChain agent which will use the enriched prompt to generate a response.
In the cell below you will instantiate an LLM model using OpenAI.

Obtain an [OpenAI API key](https://platform.openai.com/api-keys) and replace "your-openai-key" in the following cell.

In [19]:
import openai as oa
import os
from langchain_openai import ChatOpenAI


# replace with your key
os.environ["OPENAI_API_KEY"] = "your_openai_key"

# instantiate LLM model
gpt_llm = ChatOpenAI(model="gpt-4o-mini")

#create a lang chain agent that uses the system_prompt 
lc_agent = client.make_langchain_agent(llm=gpt_llm, system_prompt = "sys_prompt")

## Test it out

In the following cells you can see how the response changes based on the `user_id` and the `location` provided resulting in a personalized response for each user and based on their current location.

In [20]:
response = agent.invoke( 
    question="suggest a restaurant for tonight and tell me why you suggest it", 
    context={"user_id":"user1", "location":"Charlotte, NC"} 
)
print(response)

Hello Jim! I recommend you try *The Capital Grille*, located in Charlotte, NC. This upscale steakhouse is known for its dry-aged steaks and extensive wine list, making it a perfect choice for a special evening out. 

I suggest trying their signature *Bone-In Ribeye* or the *Filet Mignon*, both cooked to perfection. To complement your meal, consider starting with the *Lobster Bisque* or the *Prosciutto and Melon*. The ambiance is elegant and inviting, ideal for a memorable dining experience. Enjoy your evening!


In [13]:
response = agent.invoke( 
    question="suggest a restaurant for tonight and tell me why you suggest it", 
    context={"user_id":"user1", "location":"New York, NY"} 
)
print(response)

Sure, Jim! I recommend you try "The Smith," located in the East Village. It's a lively American brasserie known for its vibrant atmosphere and delicious comfort food. 

Their menu features mouthwatering options like the "The Smith Burger" with a juicy patty and their homemade pickles, or you might enjoy the "Truffle Mac & Cheese" which is a fan favorite. Don’t miss their "Roasted Chicken" served with seasonal vegetables, it's perfectly cooked and flavorful. 

The ambiance is warm and inviting, making it a great spot for a casual yet enjoyable dining experience. Enjoy your dinner!


In [14]:
response = agent.invoke( 
    question="suggest a restaurant for tonight and tell me why you suggest it", 
    context={"user_id":"user2", "location":"Charlotte, NC"} 
)
print(response)

Sure, John! I recommend trying "Mamma Ricotta's" in Charlotte, NC. This Italian restaurant is known for its warm atmosphere and delicious homemade pasta. 

One standout dish you might enjoy is the "Fettuccine Alfredo," which is creamy and rich, or you could try the "Eggplant Parmigiana," which is a customer favorite. They also have an excellent selection of wines to pair with your meal. 

The reason I suggest Mamma Ricotta's is not just for the fantastic food but also for the inviting ambiance that makes it perfect for a cozy dinner. Enjoy your evening!


In [15]:
response = agent.invoke( 
    question="suggest a restaurant for tonight and tell me why you suggest it", 
    context={"user_id":"user3", "location":"Charlotte, NC"} 
)
print(response)

Hello Jane! I recommend you try "Lang Van," a fantastic Vietnamese and Chinese restaurant located in Charlotte, NC. 

Lang Van is known for its warm atmosphere and authentic flavors. Their menu offers a range of delicious dishes, but I highly suggest trying their General Tso's Chicken and the Pho. The General Tso's Chicken is crispy and flavorful, while the Pho is aromatic and comforting, perfect for a cozy dinner.

If you're in the mood for something a bit different, their spring rolls are also a great appetizer to start your meal. The combination of delicious food and inviting ambiance makes Lang Van a perfect choice for tonight. Enjoy your dinner!


# Conclusion

Tecton prompts are used to incorporate real-time, streaming and batch features into your generative AI applications, providing a great solution for personalization. In general, it can be used to provide up to date context for any LLM driven function. 
